# The HFM library - A fast marching solver with adaptive stencils

## Part : Applications
## Chapter : Generic CPU / GPU programming

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Plots geodesics

## 0. Importing the required libraries

In [10]:
np.finfo(np.float32)

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

In [17]:
arr = np.arange(16)>3
print(arr.nbytes,arr.dtype)
print(np.array(arr,dtype='uint8').nbytes)
packed = np.packbits(arr)
print(packed.nbytes,packed.dtype)

16 bool
16
2 uint8


In [49]:
aX = np.arange(-1,2)
np.array(np.meshgrid(*(aX,)*2)).shape

(2, 3, 3)

In [41]:
type(np.float32(np.inf))

numpy.float32

In [21]:
np.iinfo(np.uint8)

iinfo(min=0, max=255, dtype=uint8)

In [29]:
import numbers
isinstance(np.float32(2),numbers.Real)

True

In [32]:
np.sign(-np.inf)

-1.0

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('TTI','FMM'))

In [2]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
#from agd.HFMUtils import HFM_CUDA
import numpy as np
import numbers
import cupy as cp
import itertools

ModuleNotFoundError: No module named 'cupy'

In [53]:
(x for x in [1,2])

<generator object <genexpr> at 0x0000021C7F95EBC8>

In [64]:
arr = np.zeros((2,),dtype='float32')
np.ones_like(np.broadcast_to(arr.flatten()[0],(2,2)))

array([[1., 1.],
       [1., 1.]], dtype=float32)

In [65]:
np.broadcast_to(np.float64(2),(2,2))

array([[2., 2.],
       [2., 2.]])

In [75]:
x=cp.array([1,2])
where = x[0]<3
x[:,where]

array([[1],
       [2]])

In [80]:
where.dtype

dtype('bool')

In [82]:
np.full_like(x,True,dtype='bool')

array([ True,  True])

In [76]:
type(x[0]<3)

cupy.core.core.ndarray

In [79]:
np.dtype('float32').type(1e-8)

1.00000001

In [54]:
type(np.zeros_like(cp.ones(2),shape=(2,2)))

TypeError: zeros_like() got an unexpected keyword argument 'shape'

In [44]:
grid=ad.array(np.meshgrid(cp.arange(2),cp.arange(2)))
print(type(grid),grid.dtype)

<class 'cupy.core.core.ndarray'> int32


In [47]:
type(cp.array([np.zeros(2),np.ones(2)]))

cupy.core.core.ndarray

In [50]:
cp.array([np.zeros(2),np.ones(2)]).dtype

dtype('float64')

In [52]:
zz= cp.zeros((2,),dtype='float32')+cp.zeros((2,),dtype='float64')
zz.dtype

dtype('float64')

In [46]:
type(cp.array(np.array([1,2,3])))

cupy.core.core.ndarray

In [42]:
np.append([0,1,2],[3])

array([0, 1, 2, 3])

In [32]:
_grid = np.meshgrid(cp.arange(2),cp.arange(2))
type(_grid[0])

cupy.core.core.ndarray

In [39]:
type(ad.array(_grid))

cupy.core.core.ndarray

In [36]:
cp.stack(_grid,axis=0)

array([[[0, 1],
        [0, 1]],

       [[0, 0],
        [1, 1]]])

In [19]:
np.zeros_like(cp.ones(2,dtype='float32')).dtype

dtype('float32')

In [22]:
cp.allclose(cp.zeros(2),cp.zeros(2))

True

In [16]:
np.__version__

'1.18.1'

In [4]:
np.zeros(2) * cp.zeros(2)

TypeError: Unsupported type <class 'numpy.ndarray'>

In [11]:
help(itertools.chain)

Help on class chain in module itertools:

class chain(builtins.object)
 |  chain(*iterables) --> chain object
 |  
 |  Return a chain object whose .__next__() method returns elements from the
 |  first iterable until it is exhausted, then elements from the next
 |  iterable, until all of the iterables are exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __setstate__(...)
 |      Set state information for unpickling.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  from_iterable(...) from builtins.type
 |      chain.from_iterable(iterable) --> chain object
 |      
 |      Alternate chain() constructor taking a single iterable argument
 |      that evaluat

In [8]:
{1:'one',2:'two'}.values()

dict_values(['one', 'two'])

In [5]:
np.float64(2)*cp.zeros(2)

array([0., 0.])

In [16]:
type(np.moveaxis(np.arange(6).reshape(2,3),0,1))

numpy.ndarray

In [23]:
np.isscalar(2.)

True

In [27]:
isinstance(np.int64(2),int)

False

In [29]:
np.int32(2)==2

True

In [34]:
isinstance(np.int8(2),numbers.Integral)

True

In [40]:
isinstance(np.array(2),numbers.Integral)

False

In [41]:
isinstance(np.float32(2.),numbers.Real)

True

In [39]:
z=np.zeros((2,2))
#zz = z.flatten()
zz=np.array(z)
zz.base is z.base

True

In [43]:
hasattr(z,'dtype')

True

In [45]:
np.empty(2)

array([4.9e-324, 1.5e-323])

In [46]:
np.minimum(np.zeros((2,2),dtype='int32'),-1)

array([[-1, -1],
       [-1, -1]], dtype=int32)

## 1.

In [3]:
hfmIn = HFMUtils.dictIn({
    'model':'Isotropic2',
    'arrayOrdering':'RowMajor',
    'seeds':[[0,0]],
    'kernel':"dummy",
    'solver':'globalIteration',
})
hfmIn.SetRect([[-1,1],[-1,1]],dimx=10)
hfmIn['cost'] = np.ones(hfmIn['dims'].astype(int),dtype='float32')

In [4]:
hfmIn

{'model': 'Isotropic2',
 'arrayOrdering': 'RowMajor',
 'seeds': [[0, 0]],
 'kernel': 'dummy',
 'solver': 'globalIteration',
 'gridScale': 0.2,
 'dims': array([10., 10.]),
 'origin': array([-1., -1.]),
 'cost': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)}

In [5]:
hfmOut = hfmIn.RunGPU(returns='in_raw') #HFM_CUDA.RunGPU(hfmIn,returns='in_raw')

Preparing the GPU kernel for (excludes compilation)
Prepating the domain data (shape,metric,...)
Preparing the values array (seeing seeds,...)
block_seedTags.shape=(2, 2, 8, 8),shape_o=array([2, 2])
{'Scalar': 'float32', 'Int': 'int32', 'shape_i': (8, 8)}


In [6]:
print(hfmOut['source'])

#define Scalar_macro
#define Int_macro
#define shape_i_macro
const int shape_i[2] = {8,8};
const int size_i=64;
const int log2_size_i=6;
typedef Scalar float;
typedef Int int;
#include "Isotropic2.h"


In [7]:
idt = np.dtype('int32')
np.dtype(idt.type)

dtype('int32')

In [8]:
hfmOut['source'][0]

'#'

In [9]:
#from agd import HFMUtils
#from agd import LinearParallel as lp
#from agd.Metrics.Seismic import Hooke,Reduced
#from agd import AutomaticDifferentiation as ad
#from agd.Plotting import savefig; #savefig.dirName = 'Images/TTI'
#norm_infinity = ad.Optimization.norm_infinity

In [10]:
import numpy as np

In [11]:
z = np.zeros(2)
type([]).__module__

'builtins'

In [12]:
np.ndarray.ctypes.c_float

AttributeError: 'getset_descriptor' object has no attribute 'c_float'

In [ ]:
z.__module__

In [ ]:
type(z).__module__

In [ ]:
np.zeros.__module__

In [ ]:
from agd import 

In [ ]:
import numpy as np
#import scipy.linalg
#from copy import copy
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
",".join(("a","b"))

In [ ]:
",".join(str(e) for e in (2,3))

In [ ]:
l=[1,2]
l.append(3)

In [ ]:
l

In [ ]:
{1:1}.get(2,3)